In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
    #for filename in filenames:
        #print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from IPython.core.magic import register_cell_magic
@register_cell_magic
def skip(line, cell=None):
    '''Skips execution of the current line/cell if line evaluates to True.'''
    if eval(line):
        return
        
    get_ipython().run_cell(cell)

In [ ]:
import pandas as pd
import tensorflow as tf
from tensorflow import keras 
from tensorflow.keras import backend as K 
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import matplotlib.pyplot as plt
import seaborn as sns
import os, cv2
import random
import dill
import gc
from sklearn.model_selection import train_test_split
import time
from kaggle_datasets import KaggleDatasets
import tensorflow_datasets.public_api as tfds
import math
from tqdm.notebook import tqdm
import tensorflow_addons as tfa
from mt_utils import *

#### Dataset links : https://www.kaggle.com/tchaye59/mt-tfrecord-custom-vocab & https://www.kaggle.com/tchaye59/mtcustomvocabimg
#### Pretraining : https://www.kaggle.com/tchaye59/mt-pretraining
#### Training: https://www.kaggle.com/tchaye59/mt-fast-distributed-training-tpu

In [ ]:
seed=123
random.seed(seed)
np.random.seed(seed)
tf.random.set_seed(seed)

In [ ]:
# NEW on TPU in TensorFlow 24: shorter cross-compatible TPU/GPU/multi-GPU/cluster-GPU detection code
tpu = None
try: # detect TPUs
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect() # TPU detection
    strategy = tf.distribute.TPUStrategy(tpu)
except ValueError: # detect GPUs
    strategy = tf.distribute.MirroredStrategy() # for GPU or multi-GPU machines
    #strategy = tf.distribute.get_strategy() # default strategy that works on CPU and single GPU
    #strategy = tf.distribute.experimental.MultiWorkerMirroredStrategy() # for clusters of multi-GPU machines

print("Number of accelerators: ", strategy.num_replicas_in_sync)


AUTO = tf.data.experimental.AUTOTUNE
REPLICAS = strategy.num_replicas_in_sync
print(f'REPLICAS: {REPLICAS}')

In [ ]:
GCS_PATH = KaggleDatasets().get_gcs_path('mtcustomvocabimg')

In [ ]:
tokenizer = CstTokenizer()
tokenizer.word_index

In [ ]:
# Initialization
max_seq = 393

# Dataset

In [ ]:
class TrainDataset(tfds.core.GeneratorBasedBuilder):
    VERSION = tfds.core.Version('0.1.0')
    
    def _split_generators(self, dl_manager):
        return [
            tfds.core.SplitGenerator(
                    name=f'train',
                    gen_kwargs={
                    },
            )
        ]
    
    def _info(self):
        return tfds.core.DatasetInfo(
            builder=self,
            description=(""),
            features=tfds.features.FeaturesDict({
                "image": tfds.features.Image(shape=(None,None,1)),
                "target": tfds.features.Tensor(shape=(max_seq,),dtype=tf.int8),
                "count": tfds.features.Tensor(dtype=tf.int32,shape=()),
            }),
        )
    
    def _generate_examples(self,**args):
        pass

In [ ]:
BATCH_SIZE = 512
STEPS_PER_TRAIN = 10
train_steps = 2424186//(BATCH_SIZE*REPLICAS)
BUFFER_SIZE = 20000

prefetch = 20
HEIGHT = 300
WIDTH = 300

In [ ]:
def data_augment(image):
    p_rotation = tf.random.uniform([], 0, 1.0, dtype=tf.float32)
    p_noise = tf.random.uniform([], 0, 1.0, dtype=tf.float32)
    p_flip1 = tf.random.uniform([], 0, 1.0, dtype=tf.float32)
    p_flip2 = tf.random.uniform([], 0, 1.0, dtype=tf.float32)
            
    # Rotation
    if p_rotation > .1:
        image = rotation(image)
        
    # Flip
    if p_flip1 > .4:
        image = tf.image.random_flip_left_right(image, seed)
        
    # Flip
    if p_flip2 > .4:
        image = tf.image.random_flip_up_down(image, seed)
        
    # Resize 
    image = tf.image.resize(image,(WIDTH, HEIGHT), method='nearest')
            
    # Noise
    if p_noise >= .4:
        image = random_noise(image)
        
    return image

def rotation(img, rotation=0.2):
    rotation = tf.random.uniform([], -1.0, 1.0, dtype=tf.float32)*rotation
    shape = tf.shape(img)
    h,w = shape[0],shape[1]
    # Pad the image with zeros to avoid losing some pixels after rotation. 
    # This will double the image width and height
    img = tf.image.pad_to_bounding_box(img,h//2, w//2,h*2, w*2)
    img = tfa.image.rotate(img,rotation,fill_value=0)
    # Now remove the zero pads
    return remove_pad(img)


def remove_pad(arr,pad_value = 0.0):
    arr_masked = tf.reduce_all(arr != pad_value , axis=-1)
    #x
    y = tf.argmax(arr_masked, axis=1)
    y = tf.where(y)
    y_min,y_max = y[0,0],y[-1,0]+1
    #y
    x = tf.argmax(arr_masked, axis=0)
    x = tf.where(x)
    x_min,x_max = x[0,0],x[-1,0]+1
    arr = arr[y_min:y_max,x_min:x_max]
    return arr

def random_noise(img,p=0.01):
    shape = tf.shape(img)
    choice = tf.random.categorical(tf.math.log([[p, 1-p]]), tf.size(img),dtype=tf.int32)
    noise = tf.random.categorical(tf.math.log([[1., 1.]]), tf.size(img),dtype=tf.int32)
    choice = tf.reshape(choice,shape)
    noise = tf.reshape(noise,shape)
    noise = tf.abs(choice-1)*noise
    choice = tf.cast(choice,img.dtype)
    noise = tf.cast(noise,img.dtype)
    return (choice*img)+noise

In [ ]:
def get_dataset(_):
    builder = TrainDataset(data_dir=GCS_PATH)
    # The following line download the dataset
    builder.download_and_prepare()
    dataset = builder.as_dataset()['train']

    # normalize, shuffle and bacth
    def preprecoss(x):
        img,target = x['image'],x['target']
        # Normalize : There are two pixels 0 and 255
        img = tf.cast(img == 0,tf.float32)
        return data_augment(img),target
    dataset = dataset.repeat().shuffle(BUFFER_SIZE).map(preprecoss,num_parallel_calls=AUTO)
    dataset = dataset.batch(BATCH_SIZE).prefetch(prefetch)
    return dataset

with strategy.scope():
    if tpu is None:
        dataset = get_dataset(0)
    else:
        dataset = strategy.experimental_distribute_datasets_from_function(get_dataset)

In [ ]:
%%time
for x,_ in get_dataset(0).take(1):
    pass
plt.imshow(x[0].numpy())

# Model 

In [ ]:
name = 'EfficientNetB0'

In [ ]:
with strategy.scope():
    image_input = tf.keras.layers.Input(shape=(WIDTH,HEIGHT,1))
    backbone_model = tf.keras.applications.EfficientNetB0(include_top=False,weights=None,input_shape=(WIDTH,HEIGHT,1),)


    backbone_model = backbone_model(image_input)
    backbone_model = tf.keras.layers.Dropout(0.3)(backbone_model)
    backbone_model = tf.keras.layers.GlobalAveragePooling2D()(backbone_model)

    output = tf.keras.layers.Dense(1,activation='linear')(backbone_model)

    model = tf.keras.Model(image_input,output)

    model.summary()

In [ ]:
with strategy.scope():
    model.compile(loss='mse',
                  experimental_steps_per_execution = STEPS_PER_TRAIN,
                  optimizer=keras.optimizers.Adam(0.001))

In [ ]:
with strategy.scope():
    save_locally = tf.saved_model.SaveOptions(experimental_io_device='/job:localhost')
    load_locally = tf.saved_model.LoadOptions(experimental_io_device='/job:localhost')
    #callbacks
    filepath=f"{name}.h5"
    callbacks_list = [
            keras.callbacks.ModelCheckpoint(filepath, 
                                            verbose=1,
                                            monitor='loss', 
                                            save_best_only=True,
                                            options=save_locally,
                                            mode='min'),
            keras.callbacks.EarlyStopping(monitor='loss',
                                          patience=20,
                                          mode='min'),
            keras.callbacks.ReduceLROnPlateau(monitor='loss',
                                              factor=0.2,
                                              patience=1,
                                              min_lr=0.00001)
    ]

In [ ]:
#! cp ../input/mt-pretraining/*.h5 .

In [ ]:
with strategy.scope():
    if os.path.exists(f'last_{name}.h5'):
        print("Loading...")
        model = tf.keras.models.load_model(f'last_{name}.h5',options=load_locally)

In [ ]:
history = model.fit(dataset,
                    steps_per_epoch=train_steps,
                    epochs=10,
                    callbacks=callbacks_list,)

In [ ]:
model.save(f'last_{name}.h5')

In [ ]:
pd.DataFrame(history.history)[['loss']].plot()